测试计算图构建

In [9]:
import torch
# 创建一些张量
x1 = torch.tensor(0.5, requires_grad=True)
x2 = torch.tensor(0.1, requires_grad=False)
y1 = torch.tensor(123.1, requires_grad=True)
y2 = torch.tensor(321.1, requires_grad=False)

# 方法1：从左到右计算 (a + b) + c
result1 = x1 + x2 + y1 + y2

# 方法3：分组计算 (a + b) + (c + d)
result3 = (x1 + x2) + (y1 + y2)

print("结果比较:")
print(f"从左到右计算: {result1.item()}")
print(f"分组计算1:    {result3.item()}")

# 计算差异
diff13 = torch.abs(result1 - result3)

print(f"\n差异:")
print(f"方法1 vs 方法3: {diff13.item():.20f}")

结果比较:
从左到右计算: 444.79998779296875
分组计算1:    444.8000183105469

差异:
方法1 vs 方法3: 0.00003051757812500000


In [8]:
import torch

# 创建一个标量张量（只包含一个元素）
tensor = torch.tensor(3.14)

# 使用 .item() 提取数值
value = tensor.item()

print(f"张量: {tensor}")      # 输出: 张量: tensor(3.1400)
print(f"数值: {value}")       # 输出: 数值: 3.14
print(f"类型: {type(value)}") # 输出: 类型: <class 'float'>

张量: 3.140000104904175
数值: 3.140000104904175
类型: <class 'float'>


In [5]:
import torch
from torchviz import make_dot

# 简单计算图测试
x = torch.randn(1, requires_grad=True)  # 删除 name 参数
x2 = torch.randn(1, requires_grad=False)  # 删除 name 参数
y = x * 2 + x2
dot = make_dot(y, params={"x": x, "x2": x2})  # torchviz 会自动使用变量名作为节点标签
dot.render("test_graph", format="png", cleanup=True)  # 生成 test_graph.png
print("✓ torchviz 工作正常！输出文件: test_graph.png")

✓ torchviz 工作正常！输出文件: test_graph.png


In [7]:
import torch
from torchviz import make_dot

def bilateral_contact_energy(
    positions: torch.Tensor,         # 节点位置 [n_nodes, 3]
    velocities: torch.Tensor,       # 节点速度 [n_nodes, 3]
    edges: torch.Tensor,            # 连接关系 [n_edges, 2] (节点索引对)
    rest_lengths: torch.Tensor,     # 弹簧自然长度 [n_edges]
    stiffness: float = 100.0,       # 弹簧刚度系数
    damping: float = 2.0,           # 阻尼系数
    contact_threshold: float = 0.5  # 接触激活阈值
) -> torch.Tensor:
    """
    计算双边接触系统的总能量（势能 + 动能）
    
    参数:
        positions: 节点位置张量 [n_nodes, 3]
        velocities: 节点速度张量 [n_nodes, 3]
        edges: 边的连接索引 [n_edges, 2]
        rest_lengths: 弹簧自然长度 [n_edges]
        stiffness: 弹簧刚度系数
        damping: 阻尼系数
        contact_threshold: 接触激活距离阈值
        
    返回:
        total_energy: 标量总能量值
    """
    # 1. 提取连接节点对
    node_i = positions[edges[:, 0]]  # [n_edges, 3]
    node_j = positions[edges[:, 1]]  # [n_edges, 3]
    
    # 2. 计算相对位移和距离
    displacement = node_j - node_i            # [n_edges, 3]
    distances = torch.linalg.norm(displacement, dim=1)  # [n_edges]
    directions = displacement / (distances.unsqueeze(1) + 1e-8)  # 单位向量 [n_edges, 3]
    
    # 3. 计算弹簧伸长量（带阈值）
    elongation = distances - rest_lengths
    active_mask = (distances < contact_threshold) | (elongation > 0)
    active_elongation = elongation * active_mask.float()
    
    # 4. 计算势能 (1/2 k x^2)
    spring_energy = 0.5 * stiffness * torch.sum(active_elongation ** 2)
    
    # 5. 计算阻尼能量 (相对速度在弹簧方向的投影)
    vel_i = velocities[edges[:, 0]]  # [n_edges, 3]
    vel_j = velocities[edges[:, 1]]  # [n_edges, 3]
    rel_velocity = vel_j - vel_i     # [n_edges, 3]
    radial_velocity = torch.sum(rel_velocity * directions, dim=1)  # [n_edges]
    damping_energy = damping * torch.sum(active_mask.float() * radial_velocity ** 2)
    
    # 6. 计算动能 (1/2 m v^2, 假设质量=1)
    kinetic_energy = 0.5 * torch.sum(velocities ** 2)
    
    # 7. 总能量
    total_energy = spring_energy + damping_energy + kinetic_energy
    return total_energy

# ===== 使用示例 =====
if __name__ == "__main__":
    torch.manual_seed(42)
    
    # 创建模拟数据
    n_nodes, n_edges = 4, 3
    positions = torch.randn(n_nodes, 3, requires_grad=True) * 0.5
    velocities = torch.randn(n_nodes, 3, requires_grad=True) * 0.1
    edges = torch.tensor([[0,1], [1,2], [2,3]])  # 链式连接
    rest_lengths = torch.tensor([1.0, 0.8, 1.2])
    
    # 计算能量
    energy = bilateral_contact_energy(
        positions, 
        velocities,
        edges,
        rest_lengths,
        stiffness=150.0,
        damping=1.5,
        contact_threshold=0.7
    )
    
    # 生成计算图（包含中间变量）
    params = {
        "positions": positions,
        "velocities": velocities,
        "rest_lengths": rest_lengths,
        "spring_energy": energy - (0.5 * torch.sum(velocities**2) + 1.5 * torch.sum(  # 分解势能
            (torch.sum((velocities[edges[:,1]] - velocities[edges[:,0]]) * 
             ((positions[edges[:,1]] - positions[edges[:,0]]) / 
              (torch.linalg.norm(positions[edges[:,1]] - positions[edges[:,0]], dim=1).unsqueeze(1) + 1e-8))
             )**2 * ((torch.linalg.norm(positions[edges[:,1]] - positions[edges[:,0]], dim=1) - rest_lengths) > -0.5).float()
        )
    }
    
    dot = make_dot(
        energy,
        params=params,
        show_attrs=True,
        show_saved=True,
        node_attr={"shape": "ellipse", "style": "filled", "fillcolor": "#E6F3FF"}
    )
    
    # 保存并渲染
    dot.render("contact_energy_graph", format="png", cleanup=True)
    print("✓ 双边接触能量计算图已生成: contact_energy_graph.png")

SyntaxError: closing parenthesis '}' does not match opening parenthesis '(' on line 86 (2700069170.py, line 92)

In [6]:
import torch
from torchviz import make_dot

torch.manual_seed(42)

# 修正 w2 的输入维度为 5（匹配拼接后的特征维度）
x = torch.randn(2, 3, requires_grad=True)
w1 = torch.randn(3, 4, requires_grad=True)
w2 = torch.randn(5, 2, requires_grad=True)  # 修改: 4 → 5
b = torch.randn(2, requires_grad=True)

# 分支1: 线性变换 + ReLU
linear1 = x @ w1           # [2,3] @ [3,4] = [2,4]
relu1 = torch.relu(linear1)
scaled_relu = relu1 * 0.5  # [2,4]

# 分支2: 二次变换
x_squared = x ** 2
norm_x = torch.linalg.norm(x_squared, dim=1, keepdim=True)  # [2,1]

# 合并分支
concat = torch.cat([scaled_relu, norm_x], dim=1)  # [2,4] + [2,1] → [2,5]

# 条件操作
mask = concat[:, 0] > 0
conditional_out = torch.where(
    mask.unsqueeze(1), 
    concat * 2, 
    concat + torch.tensor([0.1, -0.3, 0.0, 0.0, 0.0])
)  # 输出保持 [2,5]

# 主干网络（现在维度匹配）
linear2 = conditional_out @ w2  # [2,5] @ [5,2] = [2,2]
final_out = linear2 + b         # [2,2] + [2] → [2,2] (广播机制)
loss = final_out.sum() * 0.1 + torch.sigmoid(final_out).mean()

# 可视化
dot = make_dot(
    loss,
    params={
        "x": x,
        "w1": w1,
        "w2": w2,
        "b": b,
        "scaled_relu": scaled_relu,
        "norm_x": norm_x,
        "concat": concat
    }
)
dot.render("complex_graph", format="png", cleanup=True)
print("✓ 成功生成复杂计算图: complex_graph.png")

✓ 成功生成复杂计算图: complex_graph.png
